# Preprocessing Reservaciones

In [11]:
"""
This is a boilerplate pipeline 'data_processing'
generated using Kedro 0.19.12
"""
import pandas as pd

def clean_data(df: pd.DataFrame, numeric_columns: list[str]) -> pd.DataFrame:
    """
    Removes rows where:
    - 'ID_estatus_reservaciones' is between 0 and 6 (inclusive), basically
    meaning that that reservation did not arrive, was not completed among other things.
    - Any numeric column has an outlier (based on IQR).

    Args:
        df (pd.DataFrame): Input DataFrame.
        numeric_columns (list[str]): List of numeric columns to evaluate.

    Returns:
        pd.DataFrame: Cleaned DataFrame.
    """
    df_clean = df.copy()


    #df_clean = df_clean[~df_clean["ID_estatus_reservaciones"].between(0, 6)]

    outlier_mask = pd.Series([False] * len(df_clean), index=df_clean.index)

    for col in numeric_columns:
        if df_clean[col].dropna().nunique() < 2:
            continue

        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        is_outlier = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)
        outlier_mask = outlier_mask | is_outlier

    return df_clean[~outlier_mask]

import pandas as pd
import holidays

def generate_daily_and_weekly_occupancy(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Genera dos DataFrames: uno diario y otro semanal, con ocupación y variables temporales.
    """
    # Filtrar nulos y convertir tipos
    df = df.dropna(subset=["h_fec_lld", "h_fec_sda", "h_num_per", "h_num_noc", "h_tot_hab"])
    df["h_fec_lld"] = pd.to_datetime(df["h_fec_lld"], format="%Y%m%d", errors="coerce")
    df["h_fec_sda"] = pd.to_datetime(df["h_fec_sda"], format="%Y%m%d", errors="coerce")
    df["h_num_noc"] = pd.to_numeric(df["h_num_noc"], errors="coerce")
    df = df.dropna(subset=["h_fec_lld", "h_fec_sda"])
    df = df[df["h_num_noc"] == (df["h_fec_sda"] - df["h_fec_lld"]).dt.days]

    # Expandir por día
    records = []
    for _, row in df.iterrows():
        for date in pd.date_range(row["h_fec_lld"], row["h_fec_sda"] - pd.Timedelta(days=1)):
            records.append({
                "fecha": date,
                "ocupacion": row["h_num_per"],
                "cuartos": row["h_tot_hab"]
            })
    daily_df = pd.DataFrame(records)
    daily_df = daily_df.groupby("fecha", as_index=False).sum()
    daily_df = daily_df[daily_df['fecha'] <= '2020-03-13']

    # Variables temporales
    years = range(daily_df["fecha"].dt.year.min(), daily_df["fecha"].dt.year.max() + 1)
    mx_holidays = holidays.MX(years=years)
    daily_df["dia_festivo"] = daily_df["fecha"].isin(mx_holidays).astype(int)
    daily_df["semana_ano"] = daily_df["fecha"].dt.isocalendar().week
    daily_df["mes"] = daily_df["fecha"].dt.month
    daily_df["trimestre"] = daily_df["fecha"].dt.quarter
    daily_df["ano"] = daily_df["fecha"].dt.year

    # Lags
    daily_df = daily_df.sort_values("fecha")
    for lag in [1, 2, 4]:
        daily_df[f"lag_{lag}"] = daily_df["ocupacion"].shift(lag)

    # Dataset semanal
    daily_df["semana_inicio"] = daily_df["fecha"].dt.to_period("W").apply(lambda r: r.start_time)
    weekly_df = daily_df.groupby("semana_inicio", as_index=False)[["ocupacion", "cuartos"]].sum()

    # Festivos semanales
    def is_holiday_week(start_date):
        return int(any(d in mx_holidays for d in pd.date_range(start=start_date, periods=7)))

    weekly_df["dia_festivo"] = weekly_df["semana_inicio"].apply(is_holiday_week)
    weekly_df["semana_ano"] = weekly_df["semana_inicio"].dt.isocalendar().week
    weekly_df["mes"] = weekly_df["semana_inicio"].dt.month
    weekly_df["trimestre"] = weekly_df["semana_inicio"].dt.quarter
    weekly_df["ano"] = weekly_df["semana_inicio"].dt.year

    weekly_df = weekly_df.sort_values("semana_inicio")
    for lag in [1, 2, 4]:
        weekly_df[f"lag_{lag}"] = weekly_df["ocupacion"].shift(lag)
    
    daily_df = daily_df.fillna(0)
    weekly_df = weekly_df.fillna(0)


    return daily_df, weekly_df


'''

def generate_daily_occupancy(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a daily occupancy DataFrame by expanding date ranges between check-in and check-out,
    only if 'h_num_noc' (number of nights) matches the date difference.

    Args:
        df (pd.DataFrame): Input DataFrame with 'h_fec_lld', 'h_fec_sda', 'h_num_per', 'h_num_noc'.

    Returns:
        pd.DataFrame: A DataFrame with columns 'fecha' and 'ocupacion'.
    """
    df = df.dropna(subset=["h_fec_lld", "h_fec_sda", "h_num_per", "h_num_noc"])

    df["h_fec_lld"] = pd.to_datetime(df["h_fec_lld"], format="%Y%m%d", errors="coerce")
    df["h_fec_sda"] = pd.to_datetime(df["h_fec_sda"], format="%Y%m%d", errors="coerce")

    df = df.dropna(subset=["h_fec_lld", "h_fec_sda"])

    df["h_num_noc"] = pd.to_numeric(df["h_num_noc"], errors="coerce")

    df = df[df["h_num_noc"] == (df["h_fec_sda"] - df["h_fec_lld"]).dt.days]

    records = []
    for _, row in df.iterrows():
        for date in pd.date_range(row["h_fec_lld"], row["h_fec_sda"] - pd.Timedelta(days=1)):
            records.append({"fecha": date, "ocupacion": row["h_num_per"], "cuartos":row['h_tot_hab']})

    occ_df = pd.DataFrame(records)
    occ_df = occ_df.groupby("fecha", as_index=False).sum()

    return occ_df

'''



'\n\ndef generate_daily_occupancy(df: pd.DataFrame) -> pd.DataFrame:\n    """\n    Generates a daily occupancy DataFrame by expanding date ranges between check-in and check-out,\n    only if \'h_num_noc\' (number of nights) matches the date difference.\n\n    Args:\n        df (pd.DataFrame): Input DataFrame with \'h_fec_lld\', \'h_fec_sda\', \'h_num_per\', \'h_num_noc\'.\n\n    Returns:\n        pd.DataFrame: A DataFrame with columns \'fecha\' and \'ocupacion\'.\n    """\n    df = df.dropna(subset=["h_fec_lld", "h_fec_sda", "h_num_per", "h_num_noc"])\n\n    df["h_fec_lld"] = pd.to_datetime(df["h_fec_lld"], format="%Y%m%d", errors="coerce")\n    df["h_fec_sda"] = pd.to_datetime(df["h_fec_sda"], format="%Y%m%d", errors="coerce")\n\n    df = df.dropna(subset=["h_fec_lld", "h_fec_sda"])\n\n    df["h_num_noc"] = pd.to_numeric(df["h_num_noc"], errors="coerce")\n\n    df = df[df["h_num_noc"] == (df["h_fec_sda"] - df["h_fec_lld"]).dt.days]\n\n    records = []\n    for _, row in df.iterrows():

In [46]:
data_reservas = pd.read_csv('iar_Reservaciones-1.csv')

/var/folders/t1/gnq0y9p93x58rz8v61h7qpdh0000gn/T/ipykernel_21828/3727022325.py:1: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  data_reservas = pd.read_csv('iar_Reservaciones-1.csv')


In [47]:
clean_data_1 = clean_data(data_reservas,['h_num_per', 'aa_h_num_per', 'h_num_adu', 'aa_h_num_adu', 'h_num_men', 'aa_h_num_men', 'h_num_noc', 'aa_h_num_noc', 'h_tot_hab', 'aa_h_tot_hab', 'Cliente_Disp', 'aa_Cliente_Disp', 'h_tfa_total', 'aa_h_tfa_total'])

In [48]:
daily_occupancy, weekly_occupancy = generate_daily_and_weekly_occupancy(clean_data_1)

/var/folders/t1/gnq0y9p93x58rz8v61h7qpdh0000gn/T/ipykernel_21828/2204936531.py:74: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  daily_df["dia_festivo"] = daily_df["fecha"].isin(mx_holidays).astype(int)


In [49]:
daily_occupancy.tail()

,fecha,ocupacion,cuartos,dia_festivo,semana_ano,mes,trimestre,ano,lag_1,lag_2,lag_4,semana_inicio
390,2020-03-09,955,455,0,11,3,1,2020,1022.0,1311.0,879.0,2020-03-09
391,2020-03-10,929,447,0,11,3,1,2020,955.0,1022.0,1253.0,2020-03-09
392,2020-03-11,762,368,0,11,3,1,2020,929.0,955.0,1311.0,2020-03-09
393,2020-03-12,785,356,0,11,3,1,2020,762.0,929.0,1022.0,2020-03-09
394,2020-03-13,1124,499,0,11,3,1,2020,785.0,762.0,955.0,2020-03-09


In [50]:
weekly_occupancy.tail()

,semana_inicio,ocupacion,cuartos,dia_festivo,semana_ano,mes,trimestre,ano,lag_1,lag_2,lag_4
52,2020-02-10,5318,2475,0,7,2,1,2020,5929.0,7621.0,7775.0
53,2020-02-17,6866,3118,0,8,2,1,2020,5318.0,5929.0,8088.0
54,2020-02-24,6841,3146,0,9,2,1,2020,6866.0,5318.0,7621.0
55,2020-03-02,7114,3307,0,10,3,1,2020,6841.0,6866.0,5929.0
56,2020-03-09,4555,2125,0,11,3,1,2020,7114.0,6841.0,5318.0


In [51]:
daily_occupancy.to_csv("daily_occupancy.csv", index=False)
weekly_occupancy.to_csv("weekly_occupancy.csv", index=False)

# Preprocessing Platillos

In [3]:
import pandas as pd
import holidays

In [5]:
daily_occupancy = pd.read_csv("daily_occupancy.csv")

In [8]:
import pandas as pd
data_platillos = pd.read_csv('iaab_Platillos.csv')
data_ventas = pd.read_csv('iaab_Detalles_Vtas.csv')

In [9]:
data_ventas.head(50)

,ID_Detalle_vta,fecha_hoy,caja_cve,turno_cve,ID_status,fec_com,hora_alta,folio,mesero,id_operador,...,ID_empresa,ID_Map_tipo_bebyali,impto,aa_impto,Total_Sin_Impuestos,aa_total_s_impto,moneda_cve,ID_tipo_venta,ID_map_status,ID_tipo_cheque
0,223675,9/26/2018 12:00:00 AM,1,1,1,20180926,92400,12615,AL0,329,...,3,1,1,0,5.118750,0,2,0,2,1
1,223676,9/26/2018 12:00:00 AM,1,1,1,20180926,92400,12615,AL0,329,...,3,1,1,0,7.543125,0,2,0,2,1
2,223677,9/26/2018 12:00:00 AM,1,1,1,20180926,102900,12624,AL0,329,...,3,1,1,0,23.437500,0,2,0,2,2
3,223678,9/26/2018 12:00:00 AM,1,1,1,20180926,102900,12624,AL0,329,...,3,1,1,0,5.118750,0,2,0,2,2
4,223679,9/26/2018 12:00:00 AM,1,1,2,20180926,102900,12624,AL0,329,...,3,1,1,0,-5.711250,0,2,0,2,2
5,223680,9/26/2018 12:00:00 AM,1,1,1,20180926,105100,12626,AL0,329,...,3,1,1,0,12.931250,0,2,0,2,2
6,223681,9/26/2018 12:00:00 AM,1,1,2,20180926,105000,12626,AL0,329,...,3,1,1,0,-6.465625,0,2,0,2,2
7,223682,9/26/2018 12:00:00 AM,1,1,1,20180926,115400,12638,AL0,329,...,3,1,1,0,12.931250,0,2,0,2,1
8,223683,9/26/2018 12:00:00 AM,1,2,1,20180926,124800,12645,AL0,329,...,3,1,1,0,31.250000,0,2,0,2,1
9,223684,9/26/2018 12:00:00 AM,1,2,1,20180926,124800,12645,AL0,329,...,3,4,1,0,8.081875,0,2,0,2,1


In [12]:
ventas_clean = clean_data(data_ventas, ['hora_alta','HORA_MIN_SEG', 'HORA_MIN_SEG_sda','importe', 'importe_ali', 'importe_beb', 'monto_total', 'impto', 'Total_Sin_Impuestos'])

In [30]:
def generate_daily_and_weekly_demand(df_sales: pd.DataFrame, df_occupancy: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Genera dos DataFrames de demanda de platillos: uno diario y otro semanal, 
    incluyendo variables temporales, ocupación del hotel y lag features.
    También retorna un DataFrame de mapeo platillo_id ↔ platillo_cve.
    """

    df_sales = df_sales.copy()
    df_occupancy = df_occupancy.copy()

    # Asegurar formato correcto de fechas
    df_sales["fec_com"] = pd.to_datetime(df_sales["fec_com"], format="%Y%m%d", errors="coerce")
    df_occupancy["fecha"] = pd.to_datetime(df_occupancy["fecha"], errors="coerce")
    df_occupancy["semana_inicio"] = pd.to_datetime(df_occupancy["semana_inicio"], errors="coerce")

    # Filtrar ventas por fechas válidas
    valid_dates = df_occupancy["fecha"].unique()
    df_sales = df_sales[df_sales["fec_com"].isin(valid_dates)]

    # Agrupar por fecha y platillo
    grouped = df_sales.groupby(["fec_com", "platillo_cve"]).agg(
    cantidad=('platillo_cve', 'size'),
    monto_total=('monto_total', 'sum')
            ).reset_index().rename(columns={"fec_com": "fecha"})

    # Generar todas las combinaciones fecha-platillo
    all_dates = pd.date_range(grouped["fecha"].min(), grouped["fecha"].max())
    all_combinations = pd.MultiIndex.from_product([all_dates, grouped["platillo_cve"].unique()], names=["fecha", "platillo_cve"])
    daily_df = grouped.set_index(["fecha", "platillo_cve"]).reindex(all_combinations, fill_value=0).reset_index()
    daily_df = daily_df[daily_df['fecha'] <= '2020-03-13']

    # Crear diccionario platillo_id ↔ platillo_cve como DataFrame
    platillo_id_map = {cve: idx for idx, cve in enumerate(sorted(daily_df["platillo_cve"].unique()))}
    mapping_df = pd.DataFrame(list(platillo_id_map.items()), columns=["platillo_cve", "platillo_id"])

    # Codificar platillo
    daily_df["platillo_id"] = daily_df["platillo_cve"].map(platillo_id_map)

    # Unir ocupación
    daily_df = daily_df.merge(df_occupancy[["fecha", "ocupacion"]], on="fecha", how="left")

    # Variables temporales
    years = range(daily_df["fecha"].dt.year.min(), daily_df["fecha"].dt.year.max() + 1)
    mx_holidays = holidays.MX(years=years)
    daily_df["dia_festivo"] = daily_df["fecha"].isin(mx_holidays).astype(int)
    daily_df["semana_ano"] = daily_df["fecha"].dt.isocalendar().week
    daily_df["mes"] = daily_df["fecha"].dt.month
    daily_df["trimestre"] = daily_df["fecha"].dt.quarter
    daily_df["ano"] = daily_df["fecha"].dt.year
    daily_df["dia_semana"] = daily_df["fecha"].dt.weekday
    daily_df["semana_inicio"] = daily_df["fecha"].dt.to_period("W").apply(lambda r: r.start_time)

    # Agregar lags diarios
    daily_df = daily_df.sort_values(by=["platillo_id", "fecha"])
    daily_df["lag_1"] = daily_df.groupby("platillo_id")["cantidad"].shift(1)
    daily_df["lag_7"] = daily_df.groupby("platillo_id")["cantidad"].shift(7)

    # Dataset semanal
    weekly_df = daily_df.groupby(["semana_inicio", "platillo_id"], as_index=False).agg({
    "cantidad": "sum",
    "monto_total": "sum"
    })
    occupancy_weekly = df_occupancy.groupby("semana_inicio", as_index=False)["ocupacion"].sum()
    weekly_df = weekly_df.merge(occupancy_weekly, on="semana_inicio", how="left")

    # Variables temporales semanales
    weekly_df["semana_ano"] = weekly_df["semana_inicio"].dt.isocalendar().week
    weekly_df["mes"] = weekly_df["semana_inicio"].dt.month
    weekly_df["trimestre"] = weekly_df["semana_inicio"].dt.quarter
    weekly_df["ano"] = weekly_df["semana_inicio"].dt.year

    def is_holiday_week(start_date):
        return int(any(d in mx_holidays for d in pd.date_range(start=start_date, periods=7)))

    weekly_df["dia_festivo"] = weekly_df["semana_inicio"].apply(is_holiday_week)

    # Agregar lags semanales
    weekly_df = weekly_df.sort_values(by=["platillo_id", "semana_inicio"])
    weekly_df["lag_1"] = weekly_df.groupby("platillo_id")["cantidad"].shift(1)
    weekly_df["lag_2"] = weekly_df.groupby("platillo_id")["cantidad"].shift(2)

    # Eliminar platillo_cve (queda solo platillo_id)
    daily_df.drop(columns=["platillo_cve"], inplace=True)
    weekly_df.drop(columns=["platillo_cve"], errors="ignore", inplace=True)

    daily_df = daily_df.fillna(0)
    weekly_df = weekly_df.fillna(0)
    
    # 1. Revenue total por platillo
    revenue_por_platillo = weekly_df.groupby("platillo_id")["monto_total"].sum().reset_index()

    # 2. Calcular cumsum y proporción acumulada
    revenue_por_platillo = revenue_por_platillo.sort_values("monto_total", ascending=False)
    revenue_por_platillo["revenue_acum"] = revenue_por_platillo["monto_total"].cumsum()
    revenue_por_platillo["revenue_ratio"] = revenue_por_platillo["revenue_acum"] / revenue_por_platillo["monto_total"].sum()

    # 3. Filtrar top 80%
    platillos_top_80 = revenue_por_platillo[revenue_por_platillo["revenue_ratio"] <= 0.8]["platillo_id"]

    # 4. Filtrar los DataFrames
    weekly_df = weekly_df[weekly_df["platillo_id"].isin(platillos_top_80)]
    daily_df = daily_df[daily_df["platillo_id"].isin(platillos_top_80)]


    return daily_df, weekly_df, mapping_df

In [31]:
daily_demand, weekly_demand, mapping_df = generate_daily_and_weekly_demand(ventas_clean, daily_occupancy)


/var/folders/t1/gnq0y9p93x58rz8v61h7qpdh0000gn/T/ipykernel_8676/1831689973.py:45: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  daily_df["dia_festivo"] = daily_df["fecha"].isin(mx_holidays).astype(int)


In [32]:
weekly_demand.head(50)

,semana_inicio,platillo_id,cantidad,monto_total,ocupacion,semana_ano,mes,trimestre,ano,dia_festivo,lag_1,lag_2
121,2019-02-11,121,106,376.563914,5539,7,2,1,2019,0,0.0,0.0
916,2019-02-18,121,168,810.229483,6526,8,2,1,2019,0,106.0,0.0
1711,2019-02-25,121,317,7398.346711,5655,9,2,1,2019,0,168.0,106.0
2506,2019-03-04,121,534,16695.203281,6777,10,3,1,2019,0,317.0,168.0
3301,2019-03-11,121,589,19877.626029,6446,11,3,1,2019,0,534.0,317.0
4096,2019-03-18,121,350,11723.183545,6534,12,3,1,2019,1,589.0,534.0
4891,2019-03-25,121,401,13442.941425,7409,13,3,1,2019,0,350.0,589.0
5686,2019-04-01,121,493,17237.157058,6918,14,4,2,2019,0,401.0,350.0
6481,2019-04-08,121,300,11048.520247,5436,15,4,2,2019,0,493.0,401.0
7276,2019-04-15,121,426,17619.923932,7034,16,4,2,2019,0,300.0,493.0


In [36]:
daily_demand.to_csv("daily_demand.csv", index=False)
weekly_demand.to_csv("weekly_demand.csv", index=False)
mapping_df.to_csv("mapping.csv", index=False)

In [35]:
weekly_demand['platillo_id'].unique()

array([121, 124, 125, 126, 134, 140, 148, 151, 173, 174, 175, 176, 179,
       180, 284, 285, 287, 288, 289, 290, 291, 292, 293, 294, 296, 297,
       299, 308, 310, 317, 320, 330, 332, 334, 336, 345, 353, 355, 358,
       359, 365, 366, 368, 369, 416, 419, 422, 430, 432, 433, 434, 435,
       439, 446, 458, 460, 514, 527, 540, 543, 544, 545, 550, 562, 567,
       579, 591, 605, 644, 646, 648, 649, 650, 657, 658, 660, 661, 663,
       664, 667, 672, 675, 677, 678, 680, 681, 684, 700, 704, 705, 706,
       714, 715])

In [37]:
len(weekly_demand)

5301